In [14]:
import os
import requests
import zipfile
import pandas as pd

In [15]:
urls = {
    'annual_2020': 'https://www.cftc.gov/files/dea/history/dea_fut_xls_2020.zip',
    'annual_2021': 'https://www.cftc.gov/files/dea/history/dea_fut_xls_2021.zip',
    'annual_2022': 'https://www.cftc.gov/files/dea/history/dea_fut_xls_2022.zip',
    'annual_2023': 'https://www.cftc.gov/files/dea/history/dea_fut_xls_2023.zip',
    'annual_2024': 'https://www.cftc.gov/files/dea/history/dea_fut_xls_2024.zip'
}

In [16]:
extracted_dir = 'data'
os.makedirs(extracted_dir, exist_ok=True)

In [17]:
for name, url in urls.items():
    zip_file_path = f'{name}.zip'

    response = requests.get(url)
    response.raise_for_status()
    with open(zip_file_path, 'wb') as file:
        file.write(response.content)

    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        for member in zip_ref.namelist():
            filename = os.path.basename(member)
            if filename:
                extracted_file_path = os.path.join(extracted_dir, f'{name}_{filename}')
                with zip_ref.open(member) as source, open(extracted_file_path, 'wb') as target:
                    target.write(source.read())

    os.remove(zip_file_path)

print(f"The files have been downloaded and saved in the directory '{extracted_dir}'.")

The files have been downloaded and saved in the directory 'data'.


In [18]:
dfs = []
for file in os.listdir(extracted_dir):
    if file.endswith('.xls'):
        file_path = os.path.join(extracted_dir, file)
        df = pd.read_excel(file_path)
        dfs.append(df)

merged_df = pd.concat(dfs, ignore_index=True)

output_csv_path = 'data/merged_data.csv'

merged_df.to_csv(output_csv_path, index=False)

print(f"The files have been downloaded, linked and saved as '{output_csv_path}'.")

The files have been downloaded, linked and saved as 'data/merged_data.csv'.
